In [9]:
# The following code is only for Google Colab.
# If you are running this notebook locally, you should not run this cell.

# %pip install google-colab

# from google.colab import drive
# drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import zipfile

In [11]:
current_directory = os.getcwd()
current_directory

'/Users/vongoc/Documents/GitHub/ASM3-APDS'

In [12]:
with zipfile.ZipFile(f"{current_directory}/data.zip", 'r') as zip_ref:
    zip_ref.extractall(current_directory)

In [13]:
df = pd.read_csv('data/assignment3.csv')
df

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
...,...,...,...,...,...,...,...,...,...,...
19657,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
19658,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
19659,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
19660,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [14]:
# 2. Tokenizing clothing review

TOKEN_PATTERN = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?" # Token pattern for words
token_re = re.compile(TOKEN_PATTERN)

# Tokenize the review text
df["tokens"] = (
    df["Review Text"]
      .fillna("") # Replace NaN with empty string
      .astype(str)
      .apply(token_re.findall)
)

# quick spot-checks (delete when done)
print(df["tokens"].head(3).tolist())
assert isinstance(df["tokens"].iloc[0], list)

[['I', 'had', 'such', 'high', 'hopes', 'for', 'this', 'dress', 'and', 'really', 'wanted', 'it', 'to', 'work', 'for', 'me', 'i', 'initially', 'ordered', 'the', 'petite', 'small', 'my', 'usual', 'size', 'but', 'i', 'found', 'this', 'to', 'be', 'outrageously', 'small', 'so', 'small', 'in', 'fact', 'that', 'i', 'could', 'not', 'zip', 'it', 'up', 'i', 'reordered', 'it', 'in', 'petite', 'medium', 'which', 'was', 'just', 'ok', 'overall', 'the', 'top', 'half', 'was', 'comfortable', 'and', 'fit', 'nicely', 'but', 'the', 'bottom', 'half', 'had', 'a', 'very', 'tight', 'under', 'layer', 'and', 'several', 'somewhat', 'cheap', 'net', 'over', 'layers', 'imo', 'a', 'major', 'design', 'flaw', 'was', 'the', 'net', 'over', 'layer', 'sewn', 'directly', 'into', 'the', 'zipper', 'it', 'c'], ['I', 'love', 'love', 'love', 'this', 'jumpsuit', "it's", 'fun', 'flirty', 'and', 'fabulous', 'every', 'time', 'i', 'wear', 'it', 'i', 'get', 'nothing', 'but', 'great', 'compliments'], ['This', 'shirt', 'is', 'very', 'fl

In [ ]:
# 3. Lowercasing and removing words with length < 2
df["tokens"] = (
    df["Review Text"]
      .fillna("")  
      .astype(str)
      .apply(lambda text: [token.lower() for token in token_re.findall(text) if len(token) > 1])
)

# quick spot-checks 
print(df["tokens"].head(3).tolist())
assert isinstance(df["tokens"].iloc[0], list)

In [9]:
# 6. Remove the word that appears only once in the document collection, based on term frequency. 
term_freq = pd.Series(np.concatenate(df["tokens"].values)).value_counts()
term_freq

the             67690
i               45347
and             43863
a               38390
it              38181
                ...  
storeroom           1
creek               1
walnut              1
well-defined        1
chu                 1
Name: count, Length: 15402, dtype: int64

In [11]:
df["tokens"] = df["tokens"].apply(lambda tokens: [token for token in tokens if term_freq[token] > 1])
term_freq = pd.Series(np.concatenate(df["tokens"].values)).value_counts()
term_freq

the         67690
i           45347
and         43863
a           38390
it          38181
            ...  
sounded         2
oval            2
smaill          2
owls            2
artist's        2
Name: count, Length: 8348, dtype: int64

In [12]:
# 7. Remove the top 20 most frequent words based on document frequency. 
top_20_words = term_freq.nlargest(20).index
top_20_words

Index(['the', 'i', 'and', 'a', 'it', 'is', 'to', 'this', 'in', 'but', 'on',
       'for', 'of', 'with', 'was', 'so', 'my', 'dress', 'not', 'that'],
      dtype='object')

In [ ]:
df["tokens"] = df["tokens"].apply(lambda tokens: [token for token in tokens if token not in top_20_words])
term_freq = pd.Series(np.concatenate(df["tokens"].values)).value_counts()
term_freq